# 批标准化
#### 在我们正式进入模型的构建和训练之前，我们会先讲一讲数据预处理和批标准化，因为模型训练并不容易，特别是一些非常复杂的模型，并不能非常好的训练得到收敛的结果，所以对数据增加一些预处理，同时使用批标准化能够得到非常好的收敛结果，这也是卷积网络能够训练到非常深的层的一个重要原因。
### 1、数据预处理
#### 目前数据预处理最常见的方法就是中心化和标准化，中心化相当于修正数据的中心位置，实现方法非常简单，就是在每个特征维度上减去对应的均值，最后得到 0 均值的特征。标准化也非常简单，在数据变成 0 均值之后，为了使得不同的特征维度有着相同的规模，可以除以标准差近似为一个标准正态分布，也可以依据最大值和最小值将其转化为 -1 ~ 1 之间。
### 2、批标准化
#### 前面在数据预处理的时候，我们尽量输入特征不相关且满足一个标准的正态分布，这样模型的表现一般也较好。但是对于很深的网路结构，网路的非线性层会使得输出的结果变得相关，且不再满足一个标准的 N(0, 1) 的分布，甚至输出的中心已经发生了偏移，这对于模型的训练，特别是深层的模型训练非常的困难。所以在 2015 年一篇论文提出了这个方法，批标准化，简而言之，就是对于每一层网络的输出，对其做一个归一化，使其服从标准的正态分布，这样后一层网络的输入也是一个标准的正态分布，所以能够比较好的进行训练，加快收敛速度。
#### 在这里暂时不详细阐述批标准化的实现，我们只需要知道在PyTorch中如何使用批标准化。我们就以3.2节的代码为基础，在其中加入批标准化。

In [1]:
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import mnist  # 导入PyTorch中内置的mnist数据集

In [2]:
# 数据标准化，直接采用参考文档代码
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5      # 标准化，这个技巧之后会讲到
    # x = x.reshape((-1,))   # 拉平
    x = torch.from_numpy(x)  # 将numpy数组转换为tensor
    x = x.unsqueeze(0)       # 增加channel，设置为1。图片的大小为：(1, 28, 28)
    return x
# 重新加载数据集，申明定义的数据变换
train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True)
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)

In [3]:
from torch.utils.data import DataLoader
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [4]:
# 在pytorch中，实现批标准化是使用nn.BatchNorm2d(output_depth)，其中output_depth是这一层输出的图片深度。
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 第一层：数据输入是：(batch, 1, 28, 28)，经过卷积，第一层的输出则是：(batch, 25, 26, 26)
        self.layer1 = nn.Sequential(
                      nn.Conv2d(1, 25, 3),
                      nn.BatchNorm2d(25),  # 25就是输出的深度
                      nn.ReLU())
        # 第二层是池化层，池化窗口大小为(2, 2)，数据输入是：(batch, 25, 26, 26)，经过池化，第二层的输出则是：(batch, 25, 13, 13)
        # 第三层：数据输入是：(batch, 25, 13, 13)，经过卷积，第三层的输出则是：(batch, 50, 11, 11)
        self.layer3 = nn.Sequential(
                      nn.Conv2d(25, 50, 3),
                      nn.BatchNorm2d(50),
                      nn.ReLU())
        # 第四层是池化层，池化窗口大小为(2, 2)，数据输入是：(batch, 50, 11, 11)，经过池化，第四层的输出则是：(batch, 50, 5, 5)
        # 最后是线性层，共两层：
        self.fc = nn.Sequential(
                      nn.Linear(50 * 5 * 5, 1024),
                      nn.ReLU(),
                      nn.Linear(1024, 128),
                      nn.ReLU(),
                      nn.Linear(128, 10))
        self.pool = nn.MaxPool2d(2, 2)
    def forward(self, x):
        out = self.layer1(x)
        out = self.pool(out)
        out = self.layer3(out)
        out = self.pool(out)
        out = out.view(out.shape[0], -1)  # 将图片拉伸（展平），变成向量，向量的大小为 50 X 5 X 5，out.shape[0]等于batch
        out = self.fc(out)
        return out

In [5]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [6]:
# 训练模型
Epochs = 5
for epoch in range(Epochs):
    loss_sum = 0
    acc_sum = 0
    for image, label in train_data:
        image = Variable(image)
        label = Variable(label)
        # 前向传播
        y_pred = net(image)
        loss = criterion(y_pred, label)
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_sum += loss.data.item()
        _, out = y_pred.max(1)                     # 返回每一行最大值对应的下标，就是图片的预测值
        num_correct = (out == label).sum().item()  # 统计预测正确的数量
        acc = num_correct / image.shape[0]         # 得到这一个batch的平均准确率
        acc_sum += acc
    ave_train_loss = loss_sum / len(train_data)
    ave_train_acc = acc_sum / len(train_data)
    loss_sum = 0
    acc_sum = 0
    # 在测试集上检验效果
    net.eval()  # 将模型改为预测模式，eval（）时，pytorch会自动把BN和DropOut固定住，不会取平均，而是用训练好的值。
    for image, label in test_data:
        image = Variable(image)
        label = Variable(label)
        y_pred = net(image)
        loss = criterion(y_pred, label)
        loss_sum += loss.data.item()
        _, out = y_pred.max(1)
        num_correct = (out == label).sum().item()
        acc = num_correct / image.shape[0]
        acc_sum += acc
    ave_test_loss = loss_sum / len(test_data)
    ave_test_acc = acc_sum / len(test_data)
    print('epoch: {:2d}, train loss: {:.4f}, train acc: {:.4f}, test loss: {:.4f}, test acc: {:.4f}'.format(epoch + 1, ave_train_loss, ave_train_acc, ave_test_loss, ave_test_acc))

epoch:  1, train loss: 0.1731, train acc: 0.9470, test loss: 0.0466, test acc: 0.9854
epoch:  2, train loss: 0.1134, train acc: 0.9694, test loss: 0.0338, test acc: 0.9894
epoch:  3, train loss: 0.0364, train acc: 0.9887, test loss: 0.0270, test acc: 0.9920
epoch:  4, train loss: 0.0246, train acc: 0.9923, test loss: 0.0346, test acc: 0.9894
epoch:  5, train loss: 0.0179, train acc: 0.9947, test loss: 0.0297, test acc: 0.9914
